In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 144 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=292be4c21949e35c0b29a37f2ea7192ea3b1a33ccf8495bb4fd96d7e531a9730
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=9ed65b2204d84960887b5e0b3d4f010c8e3b2188b780aec97594da7692188e1c
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=fc7b65070ecdf5096dfa32093ba989d8b1044211bd9edc16028c6798b5b9ef25
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


## Imports

In [2]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import namedtuple
from sklearn import preprocessing
from bert import bert_tokenization
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.6.0
Hub version:  0.12.0


In [ ]:
df = pd.DataFrame()
df['text'] = ''
df['File No'] = ''

os.chdir(r'...\Test Dataset @ FIRE 2021\Test Dataset') #FOLDER CONTAING TEST DATA

for i in os.listdir():
    if i[-3:] == 'txt':
        f = open(i, "r", encoding='utf-8')
        dic = {}
        dic['text'] = (f.read()).replace("\n"," ")
        dic['File No'] = int(i[:-4])
        df1 = pd.DataFrame(dic, index=[0])
        df = pd.concat((df1,df), axis=0)


df = df.set_index('File No').join(labels.set_index('File No'))
df = df.rename(columns={"Real/Fake": "label"})
df.reset_index(drop=True,inplace=True)
df.to_csv(r"...", index=False) #SAVE DATASET TO CSV

In [ ]:
df = pd.DataFrame()
df['text'] = ''
df['label'] = ''

os.chdir(r'...\Training Dataset@FIRE2021\Train\Fake') #FOLDER CONTAING TRAIN DATA

for i in os.listdir():
    if i[-3:] == 'txt':
        f = open(i, "r", encoding='utf-8')
        dic = {}
        dic['text'] = (f.read()).replace("\n"," ")
        dic['label'] = 'F'
        df1 = pd.DataFrame(dic, index=[0])
        df = pd.concat((df1,df), axis=0)


os.chdir(r'...\Training Dataset@FIRE2021\Train\Real') #FOLDER CONTAING TRAIN DATA

for i in os.listdir():
    if i[-3:] == 'txt':
        f = open(i, "r", encoding='utf-8')
        dic = {}
        dic['text'] = (f.read()).replace("\n"," ")
        dic['label'] = 'R'
        df1 = pd.DataFrame(dic, index=[0])
        df = pd.concat((df1,df), axis=0)


df = df.sample(frac=1).reset_index(drop=True)
df.to_csv(r"...", index=False) #SAVE DATASET TO CSV
df.head()        

In [ ]:
df = pd.DataFrame()
df['text'] = ''
df['label'] = ''

os.chdir(r'...\Training Dataset@FIRE2021\Test\Fake')

for i in os.listdir():
    if i[-3:] == 'txt':
        f = open(i, "r", encoding='utf-8')
        dic = {}
        dic['text'] = (f.read()).replace("\n"," ")
        dic['label'] = 'F'
        df1 = pd.DataFrame(dic, index=[0])
        df = pd.concat((df1,df), axis=0)

os.chdir(r'...\Training Dataset@FIRE2021\Test\Real')

for i in os.listdir():
    if i[-3:] == 'txt':
        f = open(i, "r", encoding='utf-8')
        dic = {}
        dic['text'] = (f.read()).replace("\n"," ")
        dic['label'] = 'R'
        df1 = pd.DataFrame(dic, index=[0])
        df = pd.concat((df1,df), axis=0)

df = df.sample(frac=1).reset_index(drop=True)
df.to_csv(r"...", index=False) #SAVE DATASET TO CSV
df.head()

## Reading Dataset

In [4]:
df_train = pd.read_csv('...')
df_val = pd.read_csv("...")
df_test = pd.read_csv("...")

## Preprocessing





In [5]:
import re, string
 
def preprocessing(document):
        document = re.sub(r'[0-9]', '', document)
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        return document
  

In [6]:
df_train.text =  df_train.text.apply(preprocessing)
df_val.text =  df_val.text.apply(preprocessing)
df_test.text =  df_test.text.apply(preprocessing)

In [7]:
df_val.label.unique()

array(['R', 'F'], dtype=object)

## Label Encoding to 0 and 1

In [8]:
unique_labels = list(np.unique(df_train["label"]))

train_x = df_train["text"].values
train_y = df_train["label"].values

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_y = le.fit_transform(train_y)
train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(unique_labels), dtype='float32')

val_x = df_val["text"].values
val_y = df_val["label"].values

val_y = le.fit_transform(val_y)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=len(unique_labels), dtype='float32')

test_x = df_test["text"].values

print("number of unique labels", len(unique_labels))

number of unique labels 2


## Helper Functions

In [9]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

# Function to create attention masks
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

# Function to create segment ids
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

# Function to create input_ids, attention_masks, segment_ids for sample
def create_single_input(sentence,MAX_LEN, MAX_SEQ_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences, MAX_SEQ_LEN):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2, MAX_SEQ_LEN)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

## Downloading the MuRIL model from TFHub

In [10]:
muril_layer = hub.KerasLayer("https://tfhub.dev/google/MuRIL/1", trainable=True)

# Create tokenizer
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
max_seq_len = 512
train_x = create_input_array(train_x, max_seq_len)
val_x = create_input_array(val_x, max_seq_len)
test_x = create_input_array(test_x, max_seq_len)

100%|██████████| 262/262 [00:01<00:00, 158.13it/s]


## Defining the F1 metric

In [12]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [13]:
#downloading model

In [14]:
!pip install tf-models-official

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 352 kB 51.4 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 4.4 MB 32.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.1 MB/s 
     |████████████████████████████████| 213 kB 46.6 MB/s 
     |████████████████████████████████| 37.1 MB 49 kB/s 
     |████████████████████████████████| 99 kB 9.5 MB/s 
     |████████████████████████████████| 636 kB 50.4 MB/s 
     |████████████████████████████████| 90 kB 9.0 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=3147c1be2f0b64e77386b21e1170d9ecaae549e544ac3af59d3f5a650e1cac96
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=78eb2f549f92ba116d4dac295f73857b3c8b8cce5ae49bb32833c8abebf2c1ae
  Stored in

## Defining the model

In [15]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                    name="segment_ids")
  
outputs = muril_layer(dict(input_word_ids = input_word_ids, input_mask = input_mask, input_type_ids = segment_ids))
x = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"]) # take pooled output layer
final_output = tf.keras.layers.Dense(2, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=final_output)

    
#   optimizer = 
model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  metrics=['accuracy',f1_m])




In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'encoder_outputs':  237556225   input_mask[0][0]                 
                                                                 segment_ids[0][0]            

## Making checkpoint

In [17]:
metric = 'val_f1_m'
model_save_path='...'
import keras
callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor=metric,mode='max',save_best_only=True)]

### Training

In [18]:
num_epochs = 10

# Get the model object
history = model.fit(train_x, train_y, epochs = num_epochs, batch_size = 4, validation_data = (val_x, val_y),callbacks=callbacks)

Epoch 1/10
260/260 [==============================] - 321s 1s/step - loss: 0.6733 - accuracy: 0.5761 - f1_m: 0.5791 - val_loss: 0.6501 - val_accuracy: 0.5725 - val_f1_m: 0.5720
Epoch 2/10
260/260 [==============================] - 301s 1s/step - loss: 0.5952 - accuracy: 0.6811 - f1_m: 0.6837 - val_loss: 0.6074 - val_accuracy: 0.6756 - val_f1_m: 0.6742
Epoch 3/10
260/260 [==============================] - 300s 1s/step - loss: 0.5278 - accuracy: 0.7601 - f1_m: 0.7606 - val_loss: 0.4308 - val_accuracy: 0.8168 - val_f1_m: 0.8144
Epoch 4/10
260/260 [==============================] - 300s 1s/step - loss: 0.4913 - accuracy: 0.7823 - f1_m: 0.7804 - val_loss: 0.3937 - val_accuracy: 0.8359 - val_f1_m: 0.8321
Epoch 5/10
260/260 [==============================] - 299s 1s/step - loss: 0.3447 - accuracy: 0.8825 - f1_m: 0.8827 - val_loss: 0.3877 - val_accuracy: 0.8130 - val_f1_m: 0.8144
Epoch 6/10
260/260 [==============================] - 300s 1s/step - loss: 0.2749 - accuracy: 0.9066 - f1_m: 0.9068

In [19]:
from sklearn.metrics import classification_report
model.load_weights(model_save_path)
preds = model.predict(val_x)>0.5
print(classification_report(val_y, preds))

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       112
           1       0.86      0.92      0.89       150

   micro avg       0.87      0.87      0.87       262
   macro avg       0.87      0.86      0.87       262
weighted avg       0.87      0.87      0.87       262
 samples avg       0.87      0.87      0.87       262



In [21]:
model.save('...')

## Predictions

In [ ]:
preds = model.predict(test_x)>0.5
result = np.argmax(preds,axis=1)
result = le.inverse_transform(result)
df_test = df_test.drop('text', 1)
df_test.rename(columns={'label': 'File No', 'l': 'Real/Fake'}, inplace=True)
df_test['Real/Fake'] = [x[:1] for x in df_test['Real/Fake']]
df_test['File No'] = [x[:-4] for x in df_test['File No']]

In [ ]:
df_test.to_csv("sub1.csv", index=False)